In [1]:
import gymnasium as gym
import numpy as np
import random
from Qtabularfunctions import*
from Cartpolefamily import*

In [2]:
category="hard"
# bins=[10, 10, 10, 10]

In [13]:
bins=[2,2,2,2]

In [19]:
gen = CartPoleCategoryGenerator()
env = gen.generate_env(category)

agent = TabularQLearningAgent(
    bins=bins,
    num_actions=2,  
    lr=0.1,
    gamma=0.99,
    epsilon=0.1,
    force_mag=env.force_mag
)

Continuous action space discretized into 2 actions:
Discrete actions: [-10.  10.]


In [22]:
agent.Q

array([[[[[17.73360662, 19.4104028 ],
          [14.58421298, 19.44924965]],

         [[ 7.40955293, 20.09452023],
          [ 8.78716657, 19.95037086]]],


        [[[18.67733549, 17.73320123],
          [17.59309481, 19.37180849]],

         [[13.15792595, 19.16565849],
          [ 7.37225227, 19.78498889]]]],



       [[[[19.39845722, 14.68781331],
          [16.8403137 , 14.75330691]],

         [[17.32812051, 18.09856807],
          [14.16183497, 18.1766656 ]]],


        [[[14.09081114, 11.95084183],
          [19.86565655, 14.81338203]],

         [[17.14536827, 16.73323419],
          [17.17528232, 18.08420215]]]]])

In [21]:
episodes = 50000
for episode in range(episodes):

    state = env.reset()
    total_reward = 0
    done = False
    
    while not done:
        # Choose action (returns index 0-9)
        s=agent.disc.discretize(state)
        a=agent.choose_action(s)
        action=agent.discrete_actions[a]
        # Take action in environment
        # print(action)
        result = env.step(action)
        if len(result) == 4:
            next_state, reward, done, info = result
        else:
            next_state, reward, done, truncated, info = result
            done = done or truncated
        
        # Update Q-table
        agent.update(state, a, reward, next_state, done)
        
        state = next_state
        total_reward += reward
    
    # Decrease exploration over time
    if episode % 100 == 0:
        agent.decrease_epsilon()
    
    if episode % 100 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.3f}")

env.close()

Episode 0, Total Reward: 11.0, Epsilon: 0.099
Episode 100, Total Reward: 9.0, Epsilon: 0.098
Episode 200, Total Reward: 9.0, Epsilon: 0.097
Episode 300, Total Reward: 13.0, Epsilon: 0.096
Episode 400, Total Reward: 11.0, Epsilon: 0.095


Episode 500, Total Reward: 9.0, Epsilon: 0.094
Episode 600, Total Reward: 9.0, Epsilon: 0.093
Episode 700, Total Reward: 9.0, Epsilon: 0.092
Episode 800, Total Reward: 9.0, Epsilon: 0.091
Episode 900, Total Reward: 11.0, Epsilon: 0.090
Episode 1000, Total Reward: 11.0, Epsilon: 0.090
Episode 1100, Total Reward: 10.0, Epsilon: 0.089
Episode 1200, Total Reward: 9.0, Epsilon: 0.088
Episode 1300, Total Reward: 12.0, Epsilon: 0.087
Episode 1400, Total Reward: 9.0, Epsilon: 0.086
Episode 1500, Total Reward: 10.0, Epsilon: 0.085
Episode 1600, Total Reward: 10.0, Epsilon: 0.084
Episode 1700, Total Reward: 9.0, Epsilon: 0.083
Episode 1800, Total Reward: 10.0, Epsilon: 0.083
Episode 1900, Total Reward: 11.0, Epsilon: 0.082
Episode 2000, Total Reward: 11.0, Epsilon: 0.081
Episode 2100, Total Reward: 11.0, Epsilon: 0.080
Episode 2200, Total Reward: 10.0, Epsilon: 0.079
Episode 2300, Total Reward: 11.0, Epsilon: 0.079
Episode 2400, Total Reward: 11.0, Epsilon: 0.078
Episode 2500, Total Reward: 10.0

In [23]:
# Get different policy representations
print("1. Basic policy (state -> action index):")
basic_policy = agent.get_policy_dict()
print(f"Sample: {list(basic_policy.items())[:2]}")

print("\n2. Detailed policy:")
detailed_policy = agent.get_detailed_policy()
sample_state = list(detailed_policy.keys())[0]
print(f"Sample for state {sample_state}: {detailed_policy[sample_state]}")

# Print a nice sample
agent.print_policy_sample(num_samples=50)

1. Basic policy (state -> action index):
Sample: [((0, 0, 0, 0), np.int64(1)), ((0, 0, 0, 1), np.int64(1))]

2. Detailed policy:
Sample for state (0, 0, 0, 0): {'action_index': np.int64(1), 'continuous_action': np.float64(10.0), 'q_value': np.float64(19.41040279791221), 'all_q_values': [17.733606615757154, 19.41040279791221]}

Policy Sample (first 50 states):
State (0, 0, 0, 0) -> Action: 10.000
State (0, 0, 0, 1) -> Action: 10.000
State (0, 0, 1, 0) -> Action: 10.000
State (0, 0, 1, 1) -> Action: 10.000
State (0, 1, 0, 0) -> Action: -10.000
State (0, 1, 0, 1) -> Action: 10.000
State (0, 1, 1, 0) -> Action: 10.000
State (0, 1, 1, 1) -> Action: 10.000
State (1, 0, 0, 0) -> Action: -10.000
State (1, 0, 0, 1) -> Action: -10.000
State (1, 0, 1, 0) -> Action: 10.000
State (1, 0, 1, 1) -> Action: 10.000
State (1, 1, 0, 0) -> Action: -10.000
State (1, 1, 0, 1) -> Action: -10.000
State (1, 1, 1, 0) -> Action: -10.000
State (1, 1, 1, 1) -> Action: 10.000

Total policy size: 16 states
Unique con